<a href="https://colab.research.google.com/github/swapnilmn/Assignment_03/blob/main/Question_5_Ass_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Question 5

In [ ]:
import csv
import numpy as np
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
import time
import matplotlib.pyplot as plt
import math
!pip install wandb -qU
import wandb

# Check if CUDA is available and set the device accordingly
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def my_function():
    # Install and import the wandb library for experiment tracking
    !pip install wandb -qU
    import wandb
    
    # Login to wandb with your API key
    !wandb login 5dbfdc4a0de265f2dd69d623f169b7884aa4436f
    
    # Mount Google Drive to access files
    from google.colab import drive
    drive.mount('/content/drive')

# Call the function
my_function()


In [ ]:
import math
import time
import torch
import numpy as np

class Helper_Functions:
    @staticmethod
    def Time(s):
        # Converts seconds to minutes and seconds
        m = math.floor(s / 60)
        s -= m * 60
        return '%dm %ds' % (m, s)

    @staticmethod
    def Span(since, percent):
        # Calculates the elapsed time and estimated remaining time
        now = time.time()
        s = now - since
        es = s / percent
        rs = es - s
        return '%s (- %s)' % (Helper_Functions.Time(s), Helper_Functions.Time(rs))

    @staticmethod
    def key_printing(d, v):
        # Prints keys from a dictionary that have specific values
        f = [k for k, val in d.items() if val in v]
        if f:
            s = ''.join(f)
            if s[-1] == '\n':
                s = s[:-1]
            elif s[0] == '\t':
                s = s[1:]
        else:
            print("No keys")

        return s

    @staticmethod
    def char_acc(t, o):
        # Calculates character-level accuracy between predicted and target tensors
        with torch.no_grad():
            se = [[o[i][j].item() == t[i][j].item() for j in range(t.shape[1])] for i in range(t.shape[0])]
            c = np.sum(se)
            tc = np.sum([len(row) for row in se])
        return c / tc

    @staticmethod
    def word_acc(t, o):
        # Calculates word-level accuracy between predicted and target tensors
        o1 = torch.argmax(o, dim=1)
        with torch.no_grad():
            c = sum([(o1[i] == t[i]).sum().item() == t.shape[1] for i in range(t.shape[0])])
        return c / t.shape[0]

    @staticmethod
    def sample_equidi_pts(d, e):
        # Samples equidistant points from a given list
        s = len(d) // e
        i = np.arange(0, len(d), s)
        p = [d[j] for j in i]
        return p


HF = Helper_Functions()

# Creating a dictionary of helper functions
variables = {
    'asMinutes': HF.Time,
    'timeSince': HF.Span,
    'key_printing': HF.key_printing,
    'char_acc': HF.char_acc,
    'word_acc': HF.word_acc,
    'sample_equidi_pts': HF.sample_equidi_pts
}

# Accessing the variables
asMinutes = variables['asMinutes']
timeSince = variables['timeSince']
key_printing = variables['key_printing']
char_acc = variables['char_acc']
word_acc = variables['word_acc']
sample_equidi_pts = variables['sample_equidi_pts']

# Usage
print(asMinutes)  # Prints the formatted time in minutes and seconds
print(timeSince)  # Prints the elapsed time and estimated remaining time
print(key_printing)  # Prints keys from a dictionary that have specific values
print(char_acc)  # Calculates and returns character-level accuracy
print(word_acc)  # Calculates and returns word-level accuracy
print(sample_equidi_pts)  # Samples equidistant points from a given list


In [ ]:
# Define a dictionary of variables and their corresponding characters
variables = {
    's_t_c_r': '\t',     # Represents the tab character
    'e_d_c_r': '\n',       # Represents the newline character
    'b_k_c_r': ' ',      # Represents a blank space character
    'u_n_c_r': '\r'    # Represents a carriage return character
}

# Assign the value of 's_t_c_r' from the 'variables' dictionary to the variable 's_t_c_r'
s_t_c_r = variables['s_t_c_r']

# Assign the value of 'e_d_c_r' from the 'variables' dictionary to the variable 'e_d_c_r'
e_d_c_r = variables['e_d_c_r']

# Assign the value of 'b_k_c_r' from the 'variables' dictionary to the variable 'b_k_c_r'
b_k_c_r = variables['b_k_c_r']

# Assign the value of 'u_n_c_r' from the 'variables' dictionary to the variable 'u_n_c_r'
u_n_c_r = variables['u_n_c_r']


In [ ]:
import csv  # Import the csv module
import torch  # Import the torch module

class DataPreProcessor:
    def __init__(self):
        self.file_paths = {
            'file1': '/content/drive/MyDrive/aksharantar_sampled/mar/mar_test.csv',
            'file2': '/content/drive/MyDrive/aksharantar_sampled/mar/mar_train.csv',
            'file3': '/content/drive/MyDrive/aksharantar_sampled/mar/mar_train.csv'
        }

        self.variables = {
            's_t_c_r': '\t',      # Represents the tab character
            'e_d_c_r': '\n',        # Represents the newline character
            'b_k_c_r': ' ',       # Represents a blank space character
            'u_n_c_r': '\r'     # Represents a carriage return character
        }

    def read_csv_files(self):
        fr = {}  # Dictionary to store csv reader objects
        hd = {}  # Dictionary to store headers of each file
        dt = {}  # Dictionary to store data from each file

        for key, fp in self.file_paths.items():
            file = open(fp)  # Open the file with the given file path
            csv_reader = csv.reader(file)  # Create a csv reader object
            fr[key] = csv_reader  # Store the csv reader object in the dictionary
            hd[key] = next(csv_reader)  # Retrieve the headers from the csv reader
            dt[key] = []  # Initialize an empty list to store the data

            for row in csv_reader:
                dt[key].append(row)  # Append each row of data to the corresponding list

            file.close()  # Close the file after reading

        # Return the headers and data from each file
        return hd['file1'], hd['file2'], hd['file3'], dt['file1'], dt['file2'], dt['file3']

    def Reading_Data(self, lst):
        i = [pair[0] for pair in lst]  # Extract the first element from each pair in the given list
        t = [pair[1] for pair in lst]  # Extract the second element from each pair in the given list
        return i, t  # Return the extracted first elements as 'i' and second elements as 't'


    def Dict_lang(self, inputs, targets):
        i_dict = {}         # Dictionary to store character-to-index mapping for inputs
        max_i_length = 0    # Variable to track maximum input string length
        i_char = []         # List to store unique characters in inputs

        t_dict = {}         # Dictionary to store character-to-index mapping for targets
        max_t_length = 0    # Variable to track maximum target string length
        t_char = []         # List to store unique characters in targets

        # Encoding Inputs and updating i_dict
        for string in inputs:
            max_i_length = max(len(string), max_i_length)  # Update maximum input string length
            for char in string:
                if char not in i_dict:
                    i_dict[char] = len(i_char)   # Assign a unique index to each unique character
                    i_char.append(char)         # Store the unique character in the list

        if self.variables['b_k_c_r'] not in i_dict:
            i_dict[self.variables['b_k_c_r']] = len(i_char)   # Assign index to the blank character if not present
            i_char.append(self.variables['b_k_c_r'])

        i_dict[self.variables['u_n_c_r']] = len(i_char)     # Assign index to the unknown character
        i_char.append(self.variables['u_n_c_r'])

        if self.variables['s_t_c_r'] not in t_dict:
            t_dict[self.variables['s_t_c_r']] = len(t_char)   # Assign index to the start character if not present
            t_char.append(self.variables['s_t_c_r'])

        # Encoding Targets and updating t_dict
        for string in targets:
            max_t_length = max(len(string) + 2, max_t_length)    # Update maximum target string length
            for char in string:
                if char not in t_dict:
                    t_dict[char] = len(t_char)   # Assign a unique index to each unique character
                    t_char.append(char)         # Store the unique character in the list

        if self.variables['e_d_c_r'] not in t_dict:
            t_dict[self.variables['e_d_c_r']] = len(t_char)     # Assign index to the end character if not present
            t_char.append(self.variables['e_d_c_r'])

        if self.variables['b_k_c_r'] not in t_dict:
            t_dict[self.variables['b_k_c_r']] = len(t_char)   # Assign index to the blank character if not present
            t_char.append(self.variables['b_k_c_r'])

        return i_dict, max_i_length, i_char, t_dict, max_t_length, t_char

    def encoding_words(self, wl, ld, ml, l):
        encds = []   # List to store the encoded sequences
        for w in wl:
            encd = [ld[c] if c in ld else ld[self.variables['u_n_c_r']] for c in w]
            # Assign the index of each character in the word, or the index of unknown character if not present

            if l == 0:
                encd.extend([ld[self.variables['b_k_c_r']]] * (ml - len(encd)))
                # If 'l' is 0 (inputs), pad the sequence with blank character indices up to the maximum length
            if l == 1:
                encd = [ld[self.variables['s_t_c_r']]] + encd + [ld[self.variables['e_d_c_r']]]
                encd.extend([ld[self.variables['b_k_c_r']]] * (ml - len(encd)))
                # If 'l' is 1 (targets), add start and end character indices to the sequence and pad it with blank character indices up to the maximum length

            encds.append(encd)  # Append the encoded sequence to the list

        return encds  # Return the list of encoded sequences


    def Tokenzieations(self, train, val, test, input_dict, target_dict, max_input_length, max_target_length):
        t_i, t_t = self.Reading_Data(train)  # Reading train data
        te_i, te_t = self.Reading_Data(test)  # Reading test data
        v_i, v_t = self.Reading_Data(val)  # Reading validation data

        e_t_i = self.encoding_words(t_i, input_dict, max_input_length, 0)  # Encoding train inputs
        e_t_t = self.encoding_words(t_t, target_dict, max_target_length, 1)  # Encoding train targets
        e_v_i = self.encoding_words(v_i, input_dict, max_input_length, 0)  # Encoding validation inputs
        e_v_t = self.encoding_words(v_t, target_dict, max_target_length, 1)  # Encoding validation targets
        e_te_i = self.encoding_words(te_i, input_dict, max_input_length, 0)  # Encoding test inputs
        e_te_t = self.encoding_words(te_t, target_dict, max_target_length, 1)  # Encoding test targets

        return {
            'en_tr_ip': e_t_i,
            'en_tr_tr': e_t_t,
            'en_vl_ip': e_v_i,
            'en_vl_tr': e_v_t,
            'en_tt_ip': e_te_i,
            'en_tt_tr': e_te_t
        }

    def tensor_pair_conversion(self, tt_ip, tt_tr):
        pairs = [(torch.tensor(input_data), torch.tensor(target_data)) for input_data, target_data in
                zip(tt_ip, tt_tr)]  # Creating pairs of tensor inputs and targets
        return pairs

DPP = DataPreProcessor()

# Dictionary of functions
function_dict = {
    'tensor_pair_conversion': DPP.tensor_pair_conversion,
    'Tokenzieations': DPP.Tokenzieations,
    'encoding_words': DPP.encoding_words,
    'Reading_Data': DPP.Reading_Data,
    'Dict_lang': DPP.Dict_lang,
    'read_csv_files': DPP.read_csv_files
}

tensor_pair_conversion = function_dict['tensor_pair_conversion']
Tokenzieations = function_dict['Tokenzieations']
encoding_words = function_dict['encoding_words']
Reading_Data = function_dict['Reading_Data']
Dict_lang = function_dict['Dict_lang']
read_csv_files = function_dict['read_csv_files']

# Call the function and assign the returned values to variables
h1, h2, h3, test, val, train = read_csv_files()

tt_ip, tt_tr = Reading_Data(train)  # Reading train data
tst_ip, tst_tr = Reading_Data(test)  # Reading test data
vl_ip, vl_tr = Reading_Data(val)  # Reading validation data

print(tt_ip[1])  # Print train input at index 1
print(tt_tr[1])  # Print train target at index 1

input_dict, max_input_length, input_char, target_dict, max_target_length, target_char = Dict_lang(tt_ip + vl_ip + tst_ip, tt_tr + vl_tr + tst_tr)  # Generating dictionaries for inputs and targets

result = Tokenzieations(train, val, test, input_dict, target_dict, max_input_length, max_target_length)  # Tokenizing the data

en_tr_ip = result['en_tr_ip']  # Encoded train inputs
en_tr_tr = result['en_tr_tr']  # Encoded train targets
en_vl_ip = result['en_vl_ip']  # Encoded validation inputs
en_vl_tr = result['en_vl_tr']  # Encoded validation targets
en_tt_ip = result['en_tt_ip']  # Encoded test inputs
en_tt_tr = result['en_tt_tr']  # Encoded test targets

r = random.randint(0,100)  # Generate a random number between 0 and 100
print(key_printing(input_dict, en_tr_ip[int(r)]))  # Print the keys corresponding to the values in input_dict for the randomly selected encoded train input
print(key_printing(target_dict, en_tr_tr[int(r)]))  # Print the keys corresponding to the values in target_dict for the randomly selected encoded train target

en_tt_pa = tensor_pair_conversion(en_tr_ip, en_tr_tr)  # Convert encoded train inputs and targets into pairs of tensors
en_vl_pa = tensor_pair_conversion(en_vl_ip, en_vl_tr)  # Convert encoded validation inputs and targets into pairs of tensors
en_tst_pa = tensor_pair_conversion(en_tt_ip, en_tt_tr)  # Convert encoded test inputs and targets into pairs of tensors

pairs = (en_tt_pa, en_vl_pa, en_tst_pa)  # Combine pairs into a tuple
pair = random.choice(en_tt_pa)  # Randomly select a pair from the encoded train pairs

print(key_printing(input_dict, pair[0]))  # Print the keys corresponding to the values in input_dict for the selected pair input
print(key_printing(target_dict, pair[1]))  # Print the keys corresponding to the values in target_dict for the selected pair target


In [ ]:
class Recurrent_NN_Encoder(nn.Module):
    def __init__(self, device, cell_type, vocab_size, embed_dim, hidden_size, num_layers=1, bidirectional=False, dropout_p=0):
        super(Recurrent_NN_Encoder, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim)  # Embedding layer for converting input indices to dense vectors
        self.hidden_size = hidden_size  # Size of the hidden state in the RNN
        self.num_layers = num_layers  # Number of layers in the RNN
        self.bidirectional = bidirectional  # Flag indicating whether the RNN is bidirectional or not
        self.cell_type = cell_type  # Type of RNN cell ('lstm', 'rnn', or 'gru')
        self.dropout_p = dropout_p  # Dropout probability
        self.dropout = nn.Dropout(self.dropout_p)  # Dropout layer for regularization
        
        # Initialize the RNN cell based on the specified cell type
        if cell_type == 'lstm':
            self.rnn = nn.LSTM(embed_dim, hidden_size, num_layers=self.num_layers, batch_first=True, dropout=dropout_p, bidirectional=self.bidirectional)
        elif cell_type == 'rnn':
            self.rnn = nn.RNN(embed_dim, hidden_size, num_layers=self.num_layers, batch_first=True, dropout=dropout_p, bidirectional=self.bidirectional)
        elif cell_type == 'gru':
            self.rnn = nn.GRU(embed_dim, hidden_size, num_layers=self.num_layers, batch_first=True, dropout=dropout_p, bidirectional=self.bidirectional)

    def forward(self, x, hidden, cell):
        out = self.embedding(x)  # Perform embedding lookup to get dense representations of input indices
        out = self.dropout(out)  # Apply dropout to the input embeddings
        if self.cell_type == 'lstm':
            out, (hidden, cell) = self.rnn(out, (hidden, cell))  # Forward pass through the LSTM
            return out, hidden, cell
        elif self.cell_type == 'rnn':
            out, hidden = self.rnn(out, hidden)  # Forward pass through the RNN
            return out, hidden
        elif self.cell_type == 'gru':
            out, hidden = self.rnn(out, hidden)  # Forward pass through the GRU
            return out, hidden
    
    def init_hidden(self, batch_size):
        # Initialize the hidden and cell states with random values
        hidden = torch.randn((1 + int(self.bidirectional)) * self.num_layers, batch_size, self.hidden_size, device=device)
        cell = torch.randn((1 + int(self.bidirectional)) * self.num_layers, batch_size, self.hidden_size, device=device)
        return hidden, cell


class Recurrent_NN_Decoder(nn.Module):
    def __init__(self, device, cell_type, output_vocab, embed_size, hidden_size, max_length, dropout_p=0.1, num_layers=1, bidirectional=False):
        super(Recurrent_NN_Decoder, self).__init__()
        self.hidden_size = hidden_size  # Hidden size of the decoder
        self.output_size = output_vocab  # Size of the output vocabulary
        self.embed_size = embed_size  # Size of the input embedding
        self.dropout_p = dropout_p  # Dropout probability
        self.cell_type = cell_type  # Type of RNN cell (lstm, gru, rnn)
        self.max_length = max_length  # Maximum length of input sequence
        self.device = device  # Device (e.g., CPU or GPU) to be used for computations
        self.num_layers = num_layers  # Number of layers in the decoder
        self.embedding_decoder = nn.Embedding(self.output_size, self.embed_size)  # Embedding layer for the decoder
        self.dropout = nn.Dropout(self.dropout_p)  # Dropout layer
        self.bidirectional = bidirectional  # Flag indicating if the encoder is bidirectional

        # Determine the type of RNN cell based on the given cell_type
        if cell_type == 'lstm':
            self.rnn = nn.LSTM(self.embed_size, hidden_size, num_layers=self.num_layers, batch_first=True, bidirectional=self.bidirectional, dropout=self.dropout_p)
        elif cell_type == 'gru':
            self.rnn = nn.GRU(self.embed_size, hidden_size, num_layers=self.num_layers, batch_first=True, bidirectional=self.bidirectional, dropout=self.dropout_p)
        elif cell_type == 'rnn':
            self.rnn = nn.RNN(self.embed_size, hidden_size, num_layers=self.num_layers, batch_first=True, bidirectional=self.bidirectional, dropout=self.dropout_p)

        self.out = nn.Linear((1 + int(self.bidirectional)) * self.hidden_size, self.output_size)  # Linear layer for output prediction
        self.out_activation = nn.LogSoftmax(dim=-1)  # Log softmax activation for output probabilities

    def forward(self, input, hidden, cell):
        input = input.unsqueeze(1)  # Add a singleton dimension to the input tensor
        embedded_decoder = self.embedding_decoder(input)  # Apply embedding to the input
        embedded_decoder = self.dropout(embedded_decoder)  # Apply dropout to the embedded input

        # Pass the embedded input through the RNN cell
        if self.cell_type == 'lstm':
            output, (hidden, cell) = self.rnn(embedded_decoder, (hidden, cell))
        elif self.cell_type == 'gru':
            output, hidden = self.rnn(embedded_decoder, hidden)
        elif self.cell_type == 'rnn':
            output, hidden = self.rnn(embedded_decoder, hidden)

        output = F.relu(self.out(output))  # Apply ReLU activation to the output
        output = F.log_softmax(output, dim=-1)  # Apply log softmax activation to obtain output probabilities

        return output, hidden, cell  # Return the output, hidden state, and cell state

    def init_hidden(self, encoder_hidden, encoder_cell, encoder_bidirectional):
        hidden = encoder_hidden[-(1 + int(encoder_bidirectional)): ].repeat(self.num_layers, 1, 1)  # Initialize the hidden state using the encoder's hidden state
        cell = encoder_cell[-(1 + int(encoder_bidirectional)): ].repeat(self.num_layers, 1, 1)  # Initialize the cell state using the encoder's cell state
        return hidden, cell  # Return the initialized hidden state and cell state



class Seqence_2_Seqence_Network(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        self.encoder = encoder  # Initialize the encoder
        self.decoder = decoder  # Initialize the decoder
        self.device = device  # Store the device (e.g., CPU or GPU)
        self.max_target_length = 0  # Variable to store the maximum target sequence length
        self.sos = 0  # Start-of-sequence token
        
    def forward(self, source, target, teacher_forcing_ratio=0.5):
        batch_size = target.shape[0]  # Get the batch size
        target_len = target.shape[1]  # Get the target sequence length
        self.max_target_length = target_len  # Update the maximum target sequence length
        target_vocab_size = self.decoder.output_size  # Get the target vocabulary size
        
        outputs = torch.zeros(batch_size, target_len, target_vocab_size).to(self.device)  # Initialize the outputs tensor
        
        encoder_hidden, encoder_cell = self.encoder.init_hidden(batch_size)  # Initialize the encoder hidden state and cell state
        
        if (self.encoder.cell_type == 'lstm'):
            encoder_outputs, encoder_hidden, encoder_cell = self.encoder.forward(source, encoder_hidden, encoder_cell)
            # Forward pass through the encoder LSTM
        if (self.encoder.cell_type == 'rnn'):
            encoder_outputs, encoder_hidden = self.encoder.forward(source, encoder_hidden, encoder_cell)
            # Forward pass through the encoder RNN
        if (self.encoder.cell_type == 'gru'):
            encoder_outputs, encoder_hidden = self.encoder.forward(source, encoder_hidden, encoder_cell)
            # Forward pass through the encoder GRU
        
        input = target[:, 0]  # Set the first input to the decoder as the <sos> token
        self.sos = target[:, 0]  # Store the <sos> token
        
        hidden, cell = self.decoder.init_hidden(encoder_hidden, encoder_cell, self.encoder.bidirectional)
        # Initialize the decoder hidden state and cell state
        
        for t in range(1, target_len):
            output, hidden, cell = self.decoder.forward(input, hidden, cell)
            # Forward pass through the decoder
            outputs[:, t] = output.squeeze(1)  # Store the decoder output in the outputs tensor
            teacher_force = random.random() < teacher_forcing_ratio  # Determine whether to use teacher forcing
            top1 = output.argmax(-1)  # Get the index of the highest probability output
            input = target[:, t] if teacher_force else top1.squeeze(1)  # Set the next input to the decoder
            
        return outputs
    
    def inference(self, source, target):
        batch_size = source.shape[0]  # Get the batch size
        target_len = self.max_target_length  # Get the maximum target sequence length
        target_vocab_size = self.decoder.output_size  # Get the target vocabulary size
        
        outputs = torch.zeros(batch_size, target_len, target_vocab_size).to(self.device)  # Initialize the outputs tensor
        
        encoder_hidden, encoder_cell = self.encoder.init_hidden(batch_size)  # Initialize the encoder hidden state and cell state
        
        if (self.encoder.cell_type == 'lstm'):
            encoder_outputs, encoder_hidden, encoder_cell = self.encoder.forward(source, encoder_hidden, encoder_cell)
            # Forward pass through the encoder LSTM
        if (self.encoder.cell_type == 'rnn'):
            encoder_outputs, encoder_hidden = self.encoder.forward(source, encoder_hidden, encoder_cell)
            # Forward pass through the encoder RNN
        if (self.encoder.cell_type == 'gru'):
            encoder_outputs, encoder_hidden = self.encoder.forward(source, encoder_hidden, encoder_cell)
            # Forward pass through the encoder GRU
        
        input = self.sos  # Set the first input to the <sos> token
        
        hidden, cell = self.decoder.init_hidden(encoder_hidden, encoder_cell, self.encoder.bidirectional)
        # Initialize the decoder hidden state and cell state
        
        for t in range(1, target_len):
            output, hidden, cell = self.decoder.forward(input, hidden, cell)
            # Forward pass through the decoder
            outputs[:, t] = output.squeeze(1)  # Store the decoder output in the outputs tensor
            top1 = output.argmax(-1)  # Get the index of the highest probability output
            input = top1.squeeze(1)  # Set the next input to the decoder
            
        return outputs





In [ ]:
def vanilla_model(input_dict, target_dict):
    variables = {}
    
    # Count the number of elements in the input and target dictionaries
    variables['a'] = len(input_dict)
    variables['b'] = len(target_dict)
    
    # Set values for variables c, d, e, f, g, h, i, j, k, and max_length
    variables['c'] = 32    #batch size
    variables['d'] = 32     #val batch size
    variables['e'] = 256    #enc_embedding
    variables['f'] = 256    #dec embedding
    variables['g'] = 512    #hidden_size 
    variables['h'] = 3     #enc_number_layers
    variables['i'] = 2     #dec_num_layers
    variables['j'] = 0.4    #enc_dropout
    variables['k'] = 0.4    #dec_drop_out
    variables['max_length'] = max_target_length
    
    # Set value for variable l
    variables['l'] = 'gru'
    
    # Create an instance of Recurrent_NN_Encoder and assign it to variable m
    variables['m'] = Recurrent_NN_Encoder(device, variables['l'], variables['a'], variables['e'], variables['g'],
                                variables['h'], bidirectional=True, dropout_p=variables['j'])
    
    # Create an instance of Recurrent_NN_Decoder and assign it to variable n
    variables['n'] = Recurrent_NN_Decoder(device, variables['l'], variables['b'], variables['f'], variables['g'],
                                variables['max_length'], variables['k'], variables['i'], bidirectional=True)
    
    # Create an instance of Seqence_2_Seqence_Network by passing in the encoder and decoder instances, and assign it to variable o
    variables['o'] = Seqence_2_Seqence_Network(variables['m'], variables['n'], device).to(device)
    
    # Define a function count_parameters(model) that returns the total number of trainable parameters in a model
    def count_parameters(model):
        return sum(p.numel() for p in model.parameters() if p.requires_grad)
    
    # Print the number of trainable parameters in the Seqence_2_Seqence_Network model
    print(f' {count_parameters(variables["o"]):,} trainable parameters')
    
    # Create an instance of Adam optimizer and assign it to variable p
    variables['p'] = torch.optim.Adam(variables['o'].parameters(), lr=0.001)
    
    # Create an instance of negative log likelihood loss and assign it to variable q
    variables['q'] = nn.NLLLoss()
    
    # Return the Seqence_2_Seqence_Network model, optimizer, and criterion
    return variables['o'], variables['p'], variables['q']

# Call the vanilla_model function with input_dict and target_dict, and assign the returned values to model, optimizer, and criterion respectively
model, optimizer, criterion = vanilla_model(input_dict, target_dict)


In [ ]:
# def train_function(model, pairs, batch_size, n_iters, optimizer, tf, print_every=10, plot_every=10, log = True, Attention = False):

def train_function(m, p, b, n, o, t, p_e=10, p_v=10, l=True, A=False):
    s = time.time()  # Starting time

    p_l = []  # List to store training loss
    t_w_a = []  # List to store word-level accuracy on training set
    v_w_a = []  # List to store word-level accuracy on validation set
    p_l_t = 0  # Training loss per iteration
    p_l_v_t = 0  # Validation loss per iteration

    t_p = p[0]  # Training data
    v_p = p[1]  # Validation data
    t_a = 0  # Accumulated word-level accuracy on training set

    c = nn.NLLLoss()  # Negative log likelihood loss

    count = 0  # Iteration count
    for it in range(1, n + 1):  # Number of iterations
        for i in range(0, len(t_p) - b, b):  # Batch processing
            t_a = 0  # Reset accumulated word-level accuracy
            count += 1  # Increment iteration count

            if i + b > len(t_p):
                b = len(t_p) - i + 1  # Adjust batch size if remaining samples are less than the batch size

            i_t = torch.stack([t_p[i + j][0] for j in range(b)]).squeeze(1).long().cuda()  # Input tensor
            t_t = torch.stack([t_p[i + j][1] for j in range(b)]).squeeze(1).long().cuda()  # Target tensor

            o.zero_grad()  # Clear gradients
            out = m(i_t, t_t, teacher_forcing_ratio=t if count < 4000 else 0)  # Forward pass
            out = torch.permute(out, [0, 2, 1])  # Permute output tensor dimensions
            l = c(out, t_t)  # Calculate loss

            t_a_w = word_acc(t_t, out) * b  # Calculate word-level accuracy on training set
            t_a += t_a_w  # Accumulate word-level accuracy
            l.backward()  # Backward pass
            torch.nn.utils.clip_grad_norm_(m.parameters(), 1)  # Clip gradients to avoid exploding gradients
            o.step()  # Update model parameters

            p_l_t += l  # Accumulate training loss per iteration
            p_l_v_t += l  # Accumulate validation loss per iteration

            if count % 800 == 0:
                v_i_t = torch.stack([v_p[j][0] for j in range(b)]).squeeze(1).long().cuda()  # Validation input tensor
                v_t_t = torch.stack([v_p[j][1] for j in range(b)]).squeeze(1).long().cuda()  # Validation target tensor

                if A:
                    v_o, _ = m.inference(v_i_t, v_t_t)
                else:
                    v_o = m.inference(v_i_t, v_t_t)

                v_o = v_o.permute(0, 2, 1)  # Permute validation output tensor dimensions
                v_l = c(v_o, v_t_t)  # Calculate validation loss
                v_l_s = v_l
                wandb.log({'Val_Loss': v_l_s})  # Log validation loss

            if count % 800 == 0:
                p_l_a = p_l_t / 800  # Average training loss over 800 iterations
                p_l_t = 0  # Reset training loss accumulator
                print('%s (%d %d%%) %.7f' % (timeSince(s, it / n), it, it / n * 100, p_l_a))  # Print progress

            if count % 800 == 0:
                p_l_a = p_l_v_t / 800  # Average validation loss over 800 iterations
                p_l.append(p_l_a.detach())  # Append training loss to list
                wandb.log({'Train Loss': p_l_a})  # Log training loss
                p_l_v_t = 0  # Reset validation loss accumulator

        t_a = t_a / (len(t_p) - b)  # Calculate average word-level accuracy on training set
        t_w_a.append(t_a)  # Append word-level accuracy to list

    print(t_w_a)  # Print word-level accuracy on training set

    p_l = [l.cpu().numpy() for l in p_l]  # Convert training loss to numpy array
    p_l_s = sample_equidi_pts(p_l, n)  # Sample equidistant points from the training loss

    w_c = 0  # Total correct predictions
    for i in range(0, len(v_p) - b, b):  # Batch processing on validation set
        if i + b > len(v_p):
            b = len(v_p) - i + 1  # Adjust batch size if remaining samples are less than the batch size

        v_i_t = torch.stack([v_p[i + j][0] for j in range(b)]).squeeze(1).long().cuda()  # Validation input tensor
        v_t_t = torch.stack([v_p[i + j][1] for j in range(b)]).squeeze(1).long().cuda()  # Validation target tensor

        if A:
            v_o, _ = m.inference(v_i_t, v_t_t)
        else:
            v_o = m.inference(v_i_t, v_t_t)

        v_o = v_o.permute(0, 2, 1)  # Permute validation output tensor dimensions
        v_l = c(v_o, v_t_t)  # Calculate validation loss

        v_a_w = word_acc(v_t_t, v_o)  # Calculate word-level accuracy on validation set
        w_c += v_a_w * b  # Accumulate correct predictions

    w_a = w_c / (len(v_p) - b)  # Average word-level accuracy on validation set
    m = {'Val_Accuracy': w_a}  # Log validation accuracy
    wandb.log(m)

    print(f"Val loss = {v_l}")  # Print validation loss
    print(f'Word-level-accuracy on val set = {w_a}')  # Print word-level accuracy on validation set


In [ ]:
# Defining the sweep configuration
sw = {
    'method': 'bayes',
    'name': 'Hyperparameter Tuning-Bayesian'
}

# Defining the metric configuration
m = {
    'name': 'Val_Accuracy',
    'goal': 'maximize'
}

# Adding the metric configuration to the sweep configuration
sw['metric'] = m

# Defining the parameters configuration
p = {
    'optimiser': {'values': ['adam','nadam']},
    'teacher_forcing_ratio': {'values': [0.3, 0.5, 0.7]},
    'bidirectional': {'values': [True, False]},
    'enc_embedding': {'values': [128,256]},
    'dec_embedding': {'values': [128,256]},
    'epochs': {'values': [1]},
    'hidden_size': {'values': [64,128,256,512]},
    'enc_layers': {'values': [2,3]},
    'dec_layers': {'values': [2,3]},
    'dropout': {'values': [0.3,0.4]},
    'cell_type': {'values': ['lstm','gru','rnn']}
}

# Adding the parameters configuration to the sweep configuration
sw['parameters'] = p

# Creating a sweep using the sweep configuration and assigning it to sw_id
sw_id = wandb.sweep(sw, project="CS6910 Assignment 3")

# Function to train the sweep
def train_sweep(c=None):
    # Initializing wandb run with the given config
    with wandb.init(config=c) as r:
        c = wandb.config

        # Creating the validation data dictionary
        vd = {
            'input_dim': len(input_dict),
            'output_dim': len(target_dict),
            'batch_size': 32,
            'val_batch_size': 32,
            'enc_embedding': c.enc_embedding,
            'dec_embedding': c.dec_embedding,
            'hidden': c.hidden_size,
            'enc_num_layers': c.enc_layers,
            'dec_num_layers': c.dec_layers,
            'enc_dropout': c.dropout,
            'dec_dropout': c.dropout,
            'max_length': max_target_length,
            'cell_type': c.cell_type
        }

        # Extracting values from the validation data dictionary
        input_dim = vd['input_dim']
        output_dim = vd['output_dim']
        batch_size = vd['batch_size']
        val_batch_size = vd['val_batch_size']
        enc_embedding = vd['enc_embedding']
        dec_embedding = vd['dec_embedding']
        hidden = vd['hidden']
        enc_num_layers = vd['enc_num_layers']
        dec_num_layers = vd['dec_num_layers']
        enc_dropout = vd['enc_dropout']
        dec_dropout = vd['dec_dropout']
        max_length = vd['max_length']
        cell_type = vd['cell_type']

        # Creating the encoder and decoder models
        enc = Recurrent_NN_Encoder(device, cell_type, input_dim, enc_embedding, hidden, enc_num_layers,
                         bidirectional=c.bidirectional, dropout_p=enc_dropout)
        dec = Recurrent_NN_Decoder(device, cell_type, output_dim, dec_embedding, hidden, max_length, dec_dropout,
                         dec_num_layers, bidirectional=c.bidirectional)
        model = Seqence_2_Seqence_Network(enc, dec, device).to(device)

        # Setting the experiment name based on the config
        exp_name = f"{c.cell_type}_e_{c.optimiser}"

        # Assigning the experiment name to the wandb run
        wandb.run.name = exp_name

        # Creating the optimizer based on the chosen optimiser
        optimizer = torch.optim.Adam(model.parameters(), lr=0.001) if c.optimiser == 'adam' else torch.optim.NAdam(
            model.parameters(), lr=0.001)

        criterion = nn.NLLLoss()


        # Training the model using the train_function function
        train_function(model, pairs, 32, c.epochs, optimizer, c.teacher_forcing_ratio)
# def train_function(model, pairs, batch_size, n_iters, optimizer, tf, print_every=10, plot_every=10, log = True, Attention = False):

# Running the sweep agent with the given sweep ID and train_sweep function
wandb.agent(sw_id, train_sweep, count=1)

# Finishing the wandb run
wandb.finish()

In [ ]:
class AttentionRecurrent_NN_Decoder(nn.Module):
    def __init__(self, device, cell_type, output_vocab, embed_size, hidden_size, max_length, dropout_p=0.1, num_layers=1, bidirectional=False):
        super(AttentionRecurrent_NN_Decoder, self).__init__()
        self.hidden_size = hidden_size  # Hidden size of the decoder
        self.output_size = output_vocab  # Size of the output vocabulary
        self.embed_size = embed_size  # Size of the input embedding
        self.dropout_p = dropout_p  # Dropout probability
        self.cell_type = cell_type  # Type of RNN cell (lstm, gru, rnn)
        self.max_length = max_length  # Maximum length of input sequence
        self.device = device  # Device (e.g., CPU or GPU) to be used for computations
        self.num_layers = num_layers  # Number of layers in the decoder
        self.embedding_decoder = nn.Embedding(self.output_size, self.embed_size)  # Embedding layer for the decoder
        self.dropout = nn.Dropout(self.dropout_p)  # Dropout layer
        self.bidirectional = bidirectional  # Flag indicating if the encoder is bidirectional

        # Determine the type of RNN cell based on the given cell_type
        if cell_type == 'lstm':
            self.rnn = nn.LSTM(self.embed_size + hidden_size * (1 + int(self.bidirectional)), hidden_size, num_layers=self.num_layers, batch_first=True, bidirectional=self.bidirectional, dropout=self.dropout_p)
        elif cell_type == 'gru':
            self.rnn = nn.GRU(self.embed_size + hidden_size * (1 + int(self.bidirectional)), hidden_size, num_layers=self.num_layers, batch_first=True, bidirectional=self.bidirectional, dropout=self.dropout_p)
        elif cell_type == 'rnn':
            self.rnn = nn.RNN(self.embed_size + hidden_size * (1 + int(self.bidirectional)), hidden_size, num_layers=self.num_layers, batch_first=True, bidirectional=self.bidirectional)

        self.energy = nn.Linear(hidden_size * (2 + int(self.bidirectional)), hidden_size)  # Linear layer for energy calculation
        self.value = nn.Linear(hidden_size, 1, bias=False)  # Linear layer for value calculation
        self.softmax = nn.Softmax(dim=0)  # Softmax activation for attention weights
        self.relu = nn.ReLU()  # ReLU activation function
        self.tanh = nn.Tanh()  # Hyperbolic tangent activation function
        
        self.out = nn.Linear((1 + int(self.bidirectional)) * self.hidden_size, self.output_size)  # Linear layer for output prediction
        self.out_activation = nn.LogSoftmax(dim=-1)  # Log softmax activation for output probabilities
  
        self.hidden_reshape_linear = nn.Linear(hidden_size * 2, hidden_size)  # Linear layer for reshaping the hidden state

    def forward(self, input, encoder_states, hidden, cell):
        input = input.unsqueeze(1)  # Add a singleton dimension to the input tensor
        embedded_decoder = self.embedding_decoder(input)  # Apply embedding to the input
        embedded_decoder = self.dropout(embedded_decoder)  # Apply dropout to the embedded input

        encoder_states = encoder_states.permute(1, 0, 2)  # Permute the encoder states tensor
        sequence_length = encoder_states.shape[0]  # Obtain the sequence length
        if self.bidirectional:
            hidden_1 = self.relu(self.hidden_reshape_linear(hidden[0:2].permute(1, 0, 2).reshape(hidden.shape[1], -1))).unsqueeze(0)
        else:
            hidden_1 = hidden[0]

        hidden_reshaped = hidden_1.repeat(sequence_length, 1, 1)  # Reshape and repeat the hidden state

        energy = self.value(self.tanh(self.energy(torch.cat((hidden_reshaped, encoder_states), dim=2))))  # Compute the energy
        attention = self.softmax(energy)  # Apply softmax to obtain attention weights
        attention = attention.permute(1, 2, 0)  # Permute the attention tensor
        encoder_states = encoder_states.permute(1, 0, 2)  # Permute the encoder states tensor
        context_vector = torch.bmm(attention, encoder_states)  # Compute the context vector

        rnn_input = torch.cat((context_vector, embedded_decoder), dim=2)  # Concatenate the context vector and embedded decoder input
      
        if self.cell_type == 'lstm':
            decoder_output, (hidden, cell) = self.rnn(rnn_input, (hidden, cell))  # Pass the input through the RNN cell
        elif self.cell_type == 'gru':
            decoder_output, hidden = self.rnn(rnn_input, hidden)  # Pass the input through the RNN cell
        elif self.cell_type == 'rnn':
            decoder_output, hidden = self.rnn(rnn_input, hidden)  # Pass the input through the RNN cell

        output = F.relu(self.out(decoder_output))  # Apply ReLU activation to the output
        output = F.log_softmax(output, dim=-1)  # Apply log softmax activation to obtain output probabilities

        return output, hidden, cell, attention  # Return the output, hidden state, cell state, and attention weights

    def init_hidden(self, encoder_hidden, encoder_cell, encoder_bidirectional):
        hidden = encoder_hidden[-(1 + int(encoder_bidirectional)): ].repeat(self.num_layers, 1, 1)  # Initialize the hidden state using the encoder's hidden state
        cell = encoder_cell[-(1 + int(encoder_bidirectional)): ].repeat(self.num_layers, 1, 1)  # Initialize the cell state using the encoder's cell state
        return hidden, cell  # Return the initialized hidden state and cell state


In [ ]:
class AttentionSeqence_2_Seqence_Network(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        self.encoder = encoder  # Initialize the encoder
        self.decoder = decoder  # Initialize the decoder
        self.device = device  # Store the device (e.g., CPU or GPU)
        self.max_target_length = 0  # Variable to store the maximum target sequence length
        self.sos = 0  # Start-of-sequence token
               
    def forward(self, source, target, teacher_forcing_ratio=0.5):
        batch_size = target.shape[0]  # Get the batch size
        target_len = target.shape[1]  # Get the target sequence length
        self.max_target_length = target_len  # Update the maximum target sequence length
        target_vocab_size = self.decoder.output_size  # Get the target vocabulary size
        
        outputs = torch.zeros(batch_size, target_len, target_vocab_size).to(self.device)  # Initialize the outputs tensor

        encoder_hidden, encoder_cell = self.encoder.init_hidden(batch_size)  # Initialize the encoder hidden state and cell state

        if (self.encoder.cell_type == 'lstm'):
            encoder_outputs, encoder_hidden, encoder_cell = self.encoder.forward(source, encoder_hidden, encoder_cell)
            # Forward pass through the encoder LSTM
        if (self.encoder.cell_type == 'rnn'):
            encoder_outputs, encoder_hidden = self.encoder.forward(source, encoder_hidden, encoder_cell)
            # Forward pass through the encoder RNN
        if (self.encoder.cell_type == 'gru'):
            encoder_outputs, encoder_hidden = self.encoder.forward(source, encoder_hidden, encoder_cell)
            # Forward pass through the encoder GRU

        input = target[:, 0]  # Set the first input to the decoder as the <sos> token
        self.sos = target[:, 0]  # Store the <sos> token
        hidden, cell = self.decoder.init_hidden(encoder_hidden, encoder_cell, self.encoder.bidirectional)
        # Initialize the decoder hidden state and cell state
        
        for t in range(1, target_len):
            output, hidden, cell, _ = self.decoder.forward(input, encoder_outputs, hidden, cell)
            # Forward pass through the decoder with attention
            outputs[:, t] = output.squeeze(1)  # Store the decoder output in the outputs tensor
            teacher_force = random.random() < teacher_forcing_ratio  # Determine whether to use teacher forcing
            top1 = output.argmax(-1)  # Get the index of the highest probability output
            input = target[:, t] if teacher_force else top1.squeeze(1)  # Set the next input to the decoder
            
        return outputs

    def inference(self, source, target):
        batch_size = source.shape[0]  # Get the batch size
        target_len = self.max_target_length  # Get the maximum target sequence length
        target_vocab_size = self.decoder.output_size  # Get the target vocabulary size
        
        outputs = torch.zeros(batch_size, target_len, target_vocab_size).to(self.device)  # Initialize the outputs tensor

        encoder_hidden, encoder_cell = self.encoder.init_hidden(batch_size)  # Initialize the encoder hidden state and cell state

        if (self.encoder.cell_type == 'lstm'):
            encoder_outputs, encoder_hidden, encoder_cell = self.encoder.forward(source, encoder_hidden, encoder_cell)
            # Forward pass through the encoder LSTM
        if (self.encoder.cell_type == 'rnn'):
            encoder_outputs, encoder_hidden = self.encoder.forward(source, encoder_hidden, encoder_cell)
            # Forward pass through the encoder RNN
        if (self.encoder.cell_type == 'gru'):
            encoder_outputs, encoder_hidden = self.encoder.forward(source, encoder_hidden, encoder_cell)
            # Forward pass through the encoder GRU

        input = self.sos  # Set the first input to the <sos> token
        input_len = encoder_outputs.shape[1]  # Get the length of the encoder outputs
        hidden, cell = self.decoder.init_hidden(encoder_hidden, encoder_cell, self.encoder.bidirectional)
        attention_map = torch.zeros(batch_size, target_len, input_len)  # Initialize the attention map
        
        for t in range(1, target_len):
            output, hidden, cell, attention = self.decoder.forward(input, encoder_outputs, hidden, cell)
            # Forward pass through the decoder with attention
            attention_map[:, t - 1, :] = attention.squeeze(1)  # Store the attention weights in the attention map
            outputs[:, t] = output.squeeze(1)  # Store the decoder output in the outputs tensor
            top1 = output.argmax(-1)  # Get the index of the highest probability output
            input = top1.squeeze(1)  # Set the next input to the decoder
            
        return outputs, attention_map


In [ ]:
def AttentionModel():
    I = len(input_dict)  # Input dimension
    O = len(target_dict)  # Output dimension
    B = 32  # Batch size for training
    V = 32  # Batch size for validation
    E_e = 256  # Encoder embedding dimension
    E_d = 256  # Decoder embedding dimension
    H = 512  # Hidden dimension
    L_e = 1  # Number of layers in the encoder
    L_d = 1  # Number of layers in the decoder
    D_e = 0.25  # Dropout probability for the encoder
    D_d = 0.25  # Dropout probability for the decoder
    M = max_target_length  # Maximum target length
    C = 'lstm'  # Type of cell used in the encoder and decoder
    T = 0.7  # Teacher forcing ratio

    enc = Recurrent_NN_Encoder(device, C, I, E_e, H, L_e, bidirectional=True, dropout_p=D_e)  # Initialize the encoder
    dec = AttentionRecurrent_NN_Decoder(device, C, O, E_d, H, M, D_d, L_d, bidirectional=True)  # Initialize the decoder
    model = AttentionSeqence_2_Seqence_Network(enc, dec, device).to(device)  # Initialize the model
    optimizer = torch.optim.NAdam(model.parameters(), lr=0.001)  # Initialize the optimizer
    criterion = nn.NLLLoss()  # Initialize the criterion for loss calculation
    wandb.init(project="CS6910 Assignment 3")  # Initialize the Weights and Biases project
    train_function(model, pairs, B, 1, optimizer, T, A=True)  # Train the model

# def train_function(model, pairs, batch_size, n_iters, optimizer, tf, print_every=10, plot_every=10, log=True, Attention=False)
AttentionModel()

In [ ]:
sweep_config = {
    'method': 'bayes',  # Specifies the search method for hyperparameter optimization
    'name': 'Comparing a model with and without Attention-Bayesian'  # Name of the sweep
}

sweep_config['metric'] = {
    'name': 'Val_Accuracy',  # Name of the metric to be optimized
    'goal': 'maximize'  # Goal of the optimization (maximize or minimize)
}

sweep_config['parameters'] = {
    'optimiser': {'values': ['nadam','adam']},  # Hyperparameter: optimizer
    'teacher_forcing_ratio': {'values': [0.3,0.5,0.7]},  # Hyperparameter: teacher forcing ratio
    'bidirectional': {'values': [True, False]},  # Hyperparameter: bidirectional
    'enc_embedding': {'values': [128, 256]},  # Hyperparameter: encoder embedding size
    'dec_embedding': {'values': [128, 256]},  # Hyperparameter: decoder embedding size
    'epochs': {'values': [1]},  # Hyperparameter: number of epochs
    'hidden_size': {'values': [64,128,256,512]},  # Hyperparameter: hidden size
    'enc_layers': {'values': [2,3]},  # Hyperparameter: number of encoder layers
    'dec_layers': {'values': [2,3]},  # Hyperparameter: number of decoder layers
    'dropout': {'values': [0.25,0.3, 0.4]},  # Hyperparameter: dropout rate
    'cell_type': {'values': ['gru', 'lstm','rnn']},  # Hyperparameter: type of RNN cell
    'Attention': {'values': [True]}  # Hyperparameter: attention flag
}

sweep_id = wandb.sweep(sweep_config, project="Attention Assignment 3")  # Creates a sweep with the given configuration

def train_sweep(config=None):
    with wandb.init(config=config) as run:
        c = wandb.config  # Retrieves the configuration for the current run
        i = len(input_dict)  # Length of input dictionary
        o = len(target_dict)  # Length of target dictionary
        b = 32  # Batch size
        v = 32  # Vocabulary size
        e = c.enc_embedding  # Encoder embedding size
        d = c.dec_embedding  # Decoder embedding size
        h = c.hidden_size  # Hidden size
        en = c.enc_layers  # Number of encoder layers
        de = c.dec_layers  # Number of decoder layers
        ed = c.dropout  # Encoder dropout rate
        dd = c.dropout  # Decoder dropout rate
        l = max_target_length  # Maximum target sequence length
        t = c.cell_type  # RNN cell type

        enc = Recurrent_NN_Encoder(device, t, i, e, h, en, bidirectional=c.bidirectional, dropout_p=ed)  # Create an Recurrent_NN_Encoder instance
        if c.Attention:
            dec = AttentionRecurrent_NN_Decoder(device, t, o, d, h, l, dd, de, bidirectional=c.bidirectional)  # Create an AttentionRecurrent_NN_Decoder instance
            model = AttentionSeqence_2_Seqence_Network(enc, dec, device).to(device)  # Create an AttentionSeqence_2_Seqence_Network model
        else:
            dec = Recurrent_NN_Decoder(device, t, o, d, h, l, dd, de, bidirectional=c.bidirectional)  # Create a Recurrent_NN_Decoder instance
            model = Seqence_2_Seqence_Network(enc, dec, device).to(device)  # Create a Seqence_2_Seqence_Network model

        m = '_bi_' if c.bidirectional else '_uni_'  # Model type indicator (bidirectional or unidirectional)
        a = '_Attention' if c.Attention else '_'  # Attention indicator

        n = f"{c.cell_type}{m}e_{c.epochs}_optim_{c.optimiser}{a}"

        wandb.run.name = n  # Set the name of the run in WandB

        if c.optimiser == 'adam':
            optimizer = torch.optim.Adam(model.parameters(), lr=0.001)  # Create an Adam optimizer
        elif c.optimiser == 'nadam':
            optimizer = torch.optim.NAdam(model.parameters(), lr=0.001)  # Create a Nadam optimizer

        criterion = nn.NLLLoss()  # Define the loss criterion
        train_function(model, pairs, 32, c.epochs, optimizer, c.teacher_forcing_ratio, A=c.Attention)  # Train the model
# def train_function(model, pairs, batch_size, n_iters, optimizer, tf, print_every=10, plot_every=10, log = True, Attention = False):
wandb.agent(sweep_id, train_sweep, count=1)  # Run the sweep using the train_sweep function
wandb.finish()  # Finish the WandB run


In [ ]:
def BestAttebtionModel():
    i = len(input_dict)  # Input dimension
    o = len(target_dict)  # Output dimension
    b = v = 32  # Batch sizes
    e = 128  # Embedding dimensions
    h = 256  # Hidden layer size
    l = 128  # Number of layers
    d = 0.25  # Dropout ratios
    m = max_target_length  # Maximum target length
    c = 'gru'  # Cell type
    t = 0.7  # Teacher forcing ratio
    
    enc = Recurrent_NN_Encoder(device, c, i, e, h, l, bidirectional=True, dropout_p=d)  # Initialize encoder
    dec = AttentionRecurrent_NN_Decoder(device, c, o, e, h, m, d, l, bidirectional=True)  # Initialize decoder
    model = AttentionSeqence_2_Seqence_Network(enc, dec, device).to(device)  # Initialize model
    optimizer = torch.optim.NAdam(model.parameters(), lr=0.001)  # Initialize optimizer
    criterion = nn.NLLLoss()  # Initialize criterion
    
    wandb.init(project="Attention Assignment 3")  # Initialize wandb
    
    train_function(model, pairs, b, 1, optimizer, t, A=True)  # Train the model
BestAttebtionModel()

In [ ]:
import csv
import torch
import numpy as np

class CSVAppender:
    def __init__(self, input_dict, target_dict, max_target_length):
        # Variable assignments with single letters
        self.i = len(input_dict)  # input_dim
        self.o = len(target_dict)  # output_dim
        self.b = 32  # batch_size
        self.v = 32  # val_batch_size
        self.e1 = 128  # enc_embedding
        self.e2 = 128  # dec_embedding
        self.h = 256  # hidden
        self.el = 2  # enc_num_layers
        self.dl = 2  # dec_num_layers
        self.edo = 0.3  # enc_dropout
        self.ddo = 0.3  # dec_dropout
        self.m = max_target_length  # max_length
        self.ct = 'gru'  # cell_type
        self.tfr = 0.7  # teacher_forcing_ratio

        # Model initialization
        self.enc = Recurrent_NN_Encoder(device, self.ct, self.i, self.e1, self.h, self.el, bidirectional=True, dropout_p=self.edo)
        self.dec = AttentionRecurrent_NN_Decoder(device, self.ct, self.o, self.e2, self.h, self.m, self.ddo, self.dl, bidirectional=True)
        self.model = AttentionSeqence_2_Seqence_Network(self.enc, self.dec, device).to(device)
        self.optimizer = torch.optim.NAdam(self.model.parameters(), lr=0.001)
        self.criterion = nn.NLLLoss()

        # Initialize wandb (assuming the wandb library is imported and configured properly)
        wandb.init(project="Attention Assignment 3")

    def train(self, pairs):
        # Training iteration
        train_function(self.model, pairs, self.b, 1, self.optimizer, self.tfr, A=True)

    def append_strings_to_csv(self, file_path, string1, string2, string3):
        with open(file_path, 'a', newline='') as csvfile:
            writer = csv.writer(csvfile)
            writer.writerow([string1, string2, string3])

    def test(self, pairs):
        # pairs[2]
        tp = pairs[2]
        # test_word_count
        twc = 0
        # batch_size
        bs = 32
        # count
        c = 10
        # name
        n = 'mar'
        # file_path
        fp = '/content/drive/MyDrive/aksharantar_sampled/' + n + '.csv'

        for i in np.arange(start=0, stop=len(tp)-bs, step=bs):
            if (i + bs > len(tp)):
                bs = len(tp) - i + 1
            test_input_tensor = []
            test_target_tensor = []
            for j in range(bs):
                test_input_tensor.append(tp[i+j][0])
                test_target_tensor.append(tp[i+j][1])

            test_input_tensor = torch.stack(test_input_tensor).squeeze(1).long().cuda()
            test_target_tensor = torch.stack(test_target_tensor).squeeze(1).long().cuda()

            test_out, attention_map = self.model.inference(test_input_tensor, test_target_tensor)
            test_out = torch.permute(test_out, [0, 2, 1])
            test_loss = self.criterion(test_out, test_target_tensor)
            for j in range(bs):
                input_str = key_printing(input_dict, test_input_tensor[j])
                output_str = key_printing(target_dict, torch.argmax(test_out[j], dim=0))
                target_str = key_printing(target_dict, test_target_tensor[j])
                self.append_strings_to_csv(fp, input_str, target_str, output_str)

            test_accuracy_word = word_acc(test_target_tensor, test_out)

            twc = twc + (test_accuracy_word)*bs

        test_word_accuracy = twc / (len(tp)-bs)
        print(f"Accuracy on Test Set is {test_word_accuracy}")
        print(attention_map.shape)


model = CSVAppender(input_dict, target_dict, max_target_length)
model.train(pairs)
model.test(pairs)


In [ ]:
import torch
import numpy as np
import csv
import matplotlib.pyplot as plt
import wandb

# Import required libraries

import csv
import torch
import numpy as np

# Variable assignments with single letters
i = len(input_dict)  # input_dim
o = len(target_dict)  # output_dim
b = 32  # batch_size
v = 32  # val_batch_size
e1 = 128  # enc_embedding
e2 = 128  # dec_embedding
h = 512  # hidden
el = 2  # enc_num_layers
dl = 2  # dec_num_layers
edo = 0.3  # enc_dropout
ddo = 0.3  # dec_dropout
m = max_target_length  # max_length
ct = 'gru'  # cell_type
tfr = 0.7  # teacher_forcing_ratio

# Model initialization
enc = Recurrent_NN_Encoder(device, ct, i, e1, h, el, bidirectional=True, dropout_p=edo)  # Initialize the encoder
dec = AttentionRecurrent_NN_Decoder(device, ct, o, e2, h, m, ddo, dl, bidirectional=True)  # Initialize the decoder
model = AttentionSeqence_2_Seqence_Network(enc, dec, device).to(device)  # Initialize the sequence-to-sequence model
optimizer = torch.optim.NAdam(model.parameters(), lr=0.001)  # Initialize the optimizer
criterion = nn.NLLLoss()  # Initialize the loss function

# Initialize wandb (assuming the wandb library is imported and configured properly)
wandb.init(project="Attention-Heat-Maps")

# Training iteration
train_function(model, pairs, b, 1, optimizer, tfr, A=True)  # Perform training iterations

class SingleClass:
    def __init__(self):
        self.test_pairs = pairs[2]  # Set the test pairs
        self.test_word_count = 0  # Initialize the word count
        self.count = 10  # Set the count
        self.name = 'mar1'  # Set the name
        self.file_path = '/content/drive/MyDrive/aksharantar_sampled/' + self.name + '.csv'  # Set the file path
    
    def append_strings_to_csv(self, fp, s1, s2, s3):  # Method to append strings to a CSV file
        with open(fp, 'a', newline='') as csvfile:
            writer = csv.writer(csvfile)
            writer.writerow([s1, s2, s3])
    
    def find_index(self, tensor, req_num):  # Method to find the index of a required number in a tensor
        indices = np.where(tensor.cpu() == req_num)
        if indices[0].size > 0:
            return indices[0]+1
        else:
            return len(tensor)
    
    def pad_idx(self, inp_seq, target_seq, inp_dict, target_dict):  # Method to find the padding index
        inp_pad = inp_dict[' ']
        target_pad = target_dict[' ']
        pad_idx_inp = self.find_index(inp_seq, inp_pad)
        pad_idx_target = self.find_index(target_seq, target_pad)
        return pad_idx_inp[0], pad_idx_target[0]
    
    def run(self):  # Method to run the test
        batch_size = 32  # Set the batch size
        for i in np.arange(start=0, stop=len(self.test_pairs)-batch_size, step=batch_size):
            if (i + batch_size > len(self.test_pairs)):
                batch_size = len(self.test_pairs) - i + 1
            test_input_tensor = []
            test_target_tensor = []
            for j in range(batch_size):
                test_input_tensor.append(self.test_pairs[i+j][0])
                test_target_tensor.append(self.test_pairs[i+j][1])
  
            test_input_tensor = torch.stack(test_input_tensor).squeeze(1).long().cuda()  # Convert input to tensor and move to GPU
            test_target_tensor = torch.stack(test_target_tensor).squeeze(1).long().cuda()  # Convert target to tensor and move to GPU
  
            test_out, attention_map = model.inference(test_input_tensor, test_target_tensor)  # Perform inference
            test_out = torch.permute(test_out, [0, 2, 1])  # Permute the dimensions of test_out tensor
            test_loss = criterion(test_out, test_target_tensor)  # Calculate the loss
            inp_dict = input_dict
            for j in range(batch_size):
                inp_str = key_printing(inp_dict, test_input_tensor[j])  # Convert input tensor to string
                out_str = key_printing(target_dict, torch.argmax(test_out[j], dim=0))  # Convert output tensor to string
                target_str = key_printing(target_dict, test_target_tensor[j])  # Convert target tensor to string
            test_accuracy_word = word_acc(test_target_tensor, test_out)  # Calculate word-level accuracy
            self.test_word_count = self.test_word_count + (test_accuracy_word) * batch_size
  
        test_word_accuracy = self.test_word_count / (len(self.test_pairs) - batch_size)  # Calculate test word accuracy
        print(f"Accuracy on Test Set {test_word_accuracy}")  # Print test word accuracy
        print(attention_map.shape)  # Print the shape of attention map
  
        # print(inp_dict)  # Print the input dictionary
        # print(target_dict)  # Print the target dictionary
  
        wandb.init(project="CS6910 Assignment 3")  # Initialize wandb for logging
        for i in range(10):
            inp_seq = test_input_tensor[i+10]
            target_seq = test_target_tensor[i+10]
            attention_seq = attention_map[i+10]
            pad_idx_inp, pad_idx_target = self.pad_idx(inp_seq, target_seq, inp_dict, target_dict)
            print(pad_idx_inp, pad_idx_target)
            attention_seq_sub = attention_seq[0:int(pad_idx_inp), 0:int(pad_idx_target)]
            fig = plt.imshow(attention_seq_sub.detach().numpy())
            fig = fig.get_figure()
            row_labels = key_printing(inp_dict, inp_seq[0:pad_idx_inp])
            column_labels = key_printing(target_dict, target_seq[0:pad_idx_target])
            plt.xticks(ticks=np.arange(len(column_labels)), labels=column_labels)
            plt.yticks(ticks=np.arange(len(row_labels)), labels=row_labels)
            wandb.log({'Attention Map': fig})
            plt.show()

# Create an instance of the SingleClass
sc = SingleClass()

# Call the run method to execute the code
sc.run()
